In [1]:
!python -m pip install -qq gitpython wget                                       # install git - python and wget
# remove sample data and cloning directory if already present
!rm -Rf "sample_data" "Navarambhah"
!git clone -qq "https://github.com/MegaCreater-Projects/Navarambhah.git"        # clone repository
# move into repository
%cd Navarambhah
!git fetch                                                                      # fetch info
!git branch --all                                                               # print all branch
!git branch --all | wc -l                                                       # print total branches (-2)
# `wc -l`: This command counts the number of lines in the output, count will be two more
%cd /content
# Note: change directory command not work with not operator or with %%bash ....

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
/content/Navarambhah
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/Semantic-segmentation-using-satellite-imagery
  remotes/origin/ai-snake-game
  remotes/origin/brain-tumour-classification-n-detection
  remotes/origin/cards_detection
  remotes/origin/ct-kidney-normal-cyst-tumor-and-stone-classification
  remotes/origin/disease-classifier
  remotes/origin/eye-disease-classifiaction
  remotes/origin/food-classification
  remotes/origin/games-classification
  remotes/origin/main
  remotes/origin/multi-disease-classification
  remotes/origin/plant-disease
  remotes/origin/pneumonia-detection
  remotes/origin/rock-paper-scissor-game
  remotes/origin/rps-v2
  remotes/origin/sports-classification
  remotes/origin/text-object-detection
  remotes/origin/traffic-sigh-classification


In [2]:
import sys                                                                      # sys — System-specific parameters and functions
import os                                                                       # os — Miscellaneous operating system interfaces
import shutil                                                                   # shutil — High-level file operations
import glob                                                                     # glob — Unix style pathname pattern expansion
import wget                                                                     # source: https://pypi.org/project/wget/
# pure python download utility of linux command `wget`
if sys.version_info>=(3,6):import zipfile                                       # load for python given python version
else:import zipfile36 as zipfile                                                # else load zipfile36 , source: https://pypi.org/project/zipfile36/
# read and write ZIP files - backport of the zipfile module from Python 3.6
from git import Repo                                                            # load Repo class
try:                                                                            # try to list out all
  user_name:str='MegaCreater-Projects'                                          # user name
  repo_name:str='Navarambhah'                                                   # get repository name
  repo=Repo(repo_name)                                                          # make repository object
  branches:list=[branch for branch in repo.branches]                            # get main branchs
  remotes:list=[remote for remote in repo.remotes.origin.refs]                  # unmerge side (remote) branches
  all_branches=branches+remotes                                                 # add both
except Exception as e:                                                          # except error
  print(f'Fail to list branches. Error: {e}')                                   # make pring error
for index,branch in enumerate(all_branches):print(f'{index}: {branch}')         # make print all pranches
assert len(all_branches)==len(set(all_branches)),'Branches have same name!'     # each branch must have unique name

0: main
1: origin/HEAD
2: origin/Semantic-segmentation-using-satellite-imagery
3: origin/ai-snake-game
4: origin/brain-tumour-classification-n-detection
5: origin/cards_detection
6: origin/ct-kidney-normal-cyst-tumor-and-stone-classification
7: origin/disease-classifier
8: origin/eye-disease-classifiaction
9: origin/food-classification
10: origin/games-classification
11: origin/main
12: origin/multi-disease-classification
13: origin/plant-disease
14: origin/pneumonia-detection
15: origin/rock-paper-scissor-game
16: origin/rps-v2
17: origin/sports-classification
18: origin/text-object-detection
19: origin/traffic-sigh-classification
20: origin/vedaNetra


In [3]:
# make download all branches as zip and unzip them into repository folder
for index,branch in enumerate(all_branches):                                    # get a branch name
  # make check if name start with 'origin' and not have 'main' or 'HEAD' in it (or ends with it)
  if str(branch).startswith('origin/') and not str(branch).endswith('/main') and not str(branch).endswith('/HEAD'):# check condition
    print(f'{index}: {branch}'.ljust(75),'downloading zip ...',end=' ')         # make print info
    try:                                                                        # make try to download repo branch
      url:str=f'https://codeload.github.com/{user_name}/{repo_name}/zip/refs/heads/{str(branch).split("/",1)[-1]}'# make branch address
      downloaded_file_address:str=wget.download(url,os.path.join(repo_name,str(branch).split("/",1)[-1]),bar=None)# download branch zip
      print('done! unzipping ...',end=' ')                                      # print info
      with zipfile.ZipFile(downloaded_file_address,mode='r') as zip_reference:  # get zip file object
        zip_reference.extractall(repo_name)                                     # make unzip file to repository folder
        extracted_files:list=zip_reference.namelist()                           # get list of all zip files
      os.remove(downloaded_file_address)                                        # delete zip file
      # setup address (name) of sub branch
      sub_branch_address:str=os.path.join(os.path.dirname(downloaded_file_address),os.path.basename(downloaded_file_address).casefold())
      os.rename(os.path.join(repo_name,extracted_files[0]),sub_branch_address)  # rename file folder with repository name
      if glob.glob(os.path.join(sub_branch_address,'*','LICENSE')):             # make upgrade "LICENSE" file, if found
        if len(glob.glob(os.path.join(sub_branch_address,'*','LICENSE')))==1:   # if only single file found then only to move
          shutil.move(glob.glob(os.path.join(sub_branch_address,'*','LICENSE'))[0],os.path.join(sub_branch_address,'LICENSE'))# move and overwrite
        else:print('Failed to Update "LICENSE" (multiple files found error)...',end=' ')# print if "LICENSE" file not found
      else:print('Failed to Update "LICENSE" (file NOT found error)...',end=' ')# print if "LICENSE" file not found
      # get "README.md" file address (either "readme.md" or "README.md")
      readme_list:list=glob.glob(os.path.join(sub_branch_address,'*','readme*'))+glob.glob(os.path.join(sub_branch_address,'*','README*'))
      if readme_list:                                                           # make upgrade "README.md" file, if found
        if len(readme_list)==1:                                                 # if only single file found then only to move
          shutil.move(readme_list[0],os.path.join(sub_branch_address,'README.md'))# move and overwrite
        else:print('Failed to Update "README.md" (multiple files found error)...',end=' ')# print if "README.md" file not found
      else:print('Failed to Update "README.md" (file NOT found error)...',end=' ')# print if "LICENSE" file not found
      print('done! All done ... ')                                              # make print info
    except Exception as error:print(f'X - Failed. Error : {error}')             # if error occured
  else:print(f'{index}: {branch}'.ljust(100),'X - Failed condition NOT applied!')# if branch download failed

0: main                                                                                              X - Failed condition NOT applied!
1: origin/HEAD                                                                                       X - Failed condition NOT applied!
2: origin/Semantic-segmentation-using-satellite-imagery                     downloading zip ... done! unzipping ... done! All done ... 
3: origin/ai-snake-game                                                     downloading zip ... done! unzipping ... Failed to Update "LICENSE" (file NOT found error)... done! All done ... 
4: origin/brain-tumour-classification-n-detection                           downloading zip ... done! unzipping ... Failed to Update "README.md" (file NOT found error)... done! All done ... 
5: origin/cards_detection                                                   downloading zip ... done! unzipping ... Failed to Update "LICENSE" (file NOT found error)... Failed to Update "README.md" (file NOT found 

In [6]:
!git config --global user.name "MegaCreater"
!git config --global user.email "a.ver.archit@gmail.com"
%cd "Navarambhah"
#!git init
!git status
!git add .
!git remote remove origin
!git remote add origin "https://<token>@github.com/MegaCreater/Navarambhah.git"
!git commit -m "Initial commit"
!git branch -M main
!git push -u origin main
%cd "/content"

/content/Navarambhah
On branch main
nothing to commit, working tree clean
On branch main
nothing to commit, working tree clean
fatal: unable to access 'https://github.com@github.com/MegaCreater/Navarambhah.git/': URL using bad/illegal format or missing URL
/content


**Create a new repository on the command line**
```shell
echo "# Navarambhah" >> README.md
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/MegaCreater/Navarambhah.git
git push -u origin main
```

**Push an existing repository from the command line**
```shell
git remote add origin https://github.com/MegaCreater/Navarambhah.git
git branch -M main
git push -u origin main
```

In [7]:
# %cd "Navarambhah"
# !git remote remove origin
# !git remote add origin "https://<token>@github.com/MegaCreater/Navarambhah.git"
# !git commit -m "Initial commit"
# !git branch -M main
# !git push -u origin main
# %cd "/content"

/content/Navarambhah
On branch main
nothing to commit, working tree clean
Enumerating objects: 332, done.
Counting objects: 100% (332/332), done.
Delta compression using up to 2 threads
Compressing objects: 100% (259/259), done.
Writing objects: 100% (332/332), 123.77 MiB | 58.73 MiB/s, done.
Total 332 (delta 59), reused 293 (delta 56), pack-reused 0
remote: Resolving deltas: 100% (59/59), done.
remote: warning: See https://gh.io/lfs for more information.
remote: warning: File brain-tumour-classification-n-detection/brain_tumor_classification/training_ model/variables/variables.data-00000-of-00001 is 50.91 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
To https://github.com/MegaCreater/Navarambhah.git
 * [new branch]      main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
/content
